# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies
import gmaps
import pandas as pd
from matplotlib import pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import requests
from citipy import citipy
import time
import os

from config import gkey
gmaps.configure(api_key=gkey)
from config import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
weather_csv = "../output_data/weather_df.csv"
weather_df = pd.read_csv(weather_csv)
weather_df.head()

,Unnamed: 0,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,palmer,42.16,-72.33,77.00,36,20,11.41,1599259935
1,1,mys shmidta,62.06,48.62,48.47,95,69,5.26,1599259997
2,2,bluff,-46.60,168.33,45.00,72,1,11.01,1599259756
3,3,beringovskiy,63.05,179.32,46.54,69,100,14.99,1599259997
4,4,norman wells,35.22,-97.44,89.01,52,40,8.05,1599259997


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# store values
locations = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"].astype(float)

In [58]:
# plot heatmap
fig = gmaps.figure()

# create layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=200, point_radius=5)

#add layer to map
fig.add_layer(heat_layer)

#display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
ideal_df = weather_df.loc[(weather_df["Max Temp"] > 65) & (weather_df['Max Temp'] < 77) & (weather_df["Cloudiness"] == 0) & (weather_df['Humidity'] < 50) &(weather_df['Wind Speed'] < 15), :]

In [66]:
ideal_df = ideal_df.dropna(how = 'any')

In [67]:
ideal_df.reset_index(inplace=True)
ideal_df.head()

,index,Unnamed: 0,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,47,47,faya,18.39,42.45,68.00,30,0,5.35,1599259983
1,120,120,korla,41.76,86.15,73.00,45,0,1.99,1599259924
2,307,307,airai,-8.93,125.41,65.08,42,0,3.38,1599260082
3,312,312,komsomolskiy,40.43,71.72,66.20,48,0,6.93,1599260104
4,376,376,kentau,43.52,68.51,65.03,26,0,10.92,1599260118


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [46]:
# Add marker layer ontop of heat map


# Display figure
